## Initial configuration

In [18]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import cv2
import gc
from tqdm import tqdm,trange

import torch
from torchsummary import summary
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt

import urllib.request
import zipfile
import json

In [19]:
#Usefull for better outputs visualization
PURPLE = "\033[95m"
CYAN = "\033[96m"
DARKCYAN = "\033[36m"
BLUE = "\033[94m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
END = "\033[0m"

### Project settings

In [131]:
data_dir_unsplitted = "dataset/unsplitted"
save_dir_splitted = "dataset/splitted"
data_dir = "dataset/splitted"
models_dir ="models/" #the resuls will be saved here

os.makedirs(models_dir,exist_ok=True)


patch_size = 128
n_samples = 40
batch_size = 10


grayscale= False
resize_to= None
fft = False


learning_rate_e = 0.001
learning_rate_d = 0.001
#learning_rate_c = 0.00001
num_epochs = 50


#@DEPRECATED VARIABLE
ALREADY_SPLITTED = True #-> we will use only "LOAD_DATASET"


LOAD_DATASET = True #if true "ALREADY_SPLITTED" is ignored

#to be implemented:

DATASET_ALREADY_DOWNLOADED = True 
DATASET_ALREADY_UNZIPPED = True

LOAD_MODEL = False 
MODEL_ALREADY_DOWNLOADED = True 

## Dataset preparation

### Dataset splitting and data augmentation

In [132]:
classes = ["CLL", "FL", "MCL"]
id_to_name = {0:"CLL",1:"FL",2:"MCL"}

if(not ALREADY_SPLITTED or not LOAD_DATASET):
    data = []
    labels = []
    for i, cl in enumerate(classes):
        path = os.path.join(data_dir_unsplitted, cl)
        images = os.listdir(path)
        for img in tqdm(images,desc=cl):
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path).astype(np.uint8)
            
            data.append(img)
            labels.append(i) 


    data = np.array(data)
    labels = np.array(labels)


    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


    train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.125, random_state=42)

    print("Train samples:", len(train_data))
    print("Test samples:", len(test_data))
    print("Validation samples:", len(val_data))

### Saving the splitted dataset

In [133]:
def crop_image(image, patch_size, n_samples = 40):
    h, w, _ = image.shape
    h_eff = h - patch_size
    w_eff = w - patch_size
    samples = []
    
    for i in range(n_samples):
        h_vertex = np.random.randint(h_eff)
        w_vertex = np.random.randint(w_eff)
        samples.append(image[h_vertex:h_vertex+patch_size, w_vertex:w_vertex+patch_size, :])
    
    samples = np.stack(samples)
    return samples   

In [134]:
#SAVING THE DATASET
if(not ALREADY_SPLITTED or not LOAD_DATASET):
    #TRAIN
    os.makedirs(os.path.join(save_dir_splitted,"train","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"train","images_patch"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"train","labels"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"train","labels_patch"),exist_ok=True)
     
    train_labels_dict={}
    train_labels_patch_dict={}
    
    for i, image in enumerate(tqdm(train_data,desc = "Train")):                       #images
        file_name = str(i)+"_train"+".png"
        
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        train_labels_dict[file_name] = int(train_labels[i])
        cv2.imwrite(path,image.astype(np.float64))
        
        image_patch = crop_image(image, patch_size, n_samples)
        for j in range(len(image_patch)):
            file_name = str(i)+"-"+str(j)+"_train"+".png"
            path = os.path.join(save_dir_splitted,"train","images_patch",file_name)
            train_labels_patch_dict[file_name] = int(train_labels[i])
            cv2.imwrite(path,image_patch[j].astype(np.float64))
       
    
    
    with open(os.path.join(save_dir_splitted,"train","labels","train.json"), 'w') as f:
        json.dump(train_labels_dict, f)#labels
    with open(os.path.join(save_dir_splitted,"train","labels_patch","train_patch.json"), 'w') as f:
        json.dump(train_labels_patch_dict, f)#labels

    #TEST
    os.makedirs(os.path.join(save_dir_splitted,"test","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"test","images_patch"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"test","labels"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"test","labels_patch"),exist_ok=True)

    test_labels_dict={}
    test_labels_patch_dict={}
    
    for i, image in enumerate(tqdm(test_data,desc = "Test")):                      #images
        file_name = str(i)+"_test"+".png"
        path = os.path.join(save_dir_splitted,"test","images",file_name)
        
        cv2.imwrite(path,image.astype(np.float64))
        test_labels_dict[file_name] = int(test_labels[i]) 
        
        image_patch = crop_image(image, patch_size, n_samples)
        for j in range(len(image_patch)):
            file_name = str(i)+"-"+str(j)+"_test"+".png"
            path = os.path.join(save_dir_splitted,"test","images_patch",file_name)
            test_labels_patch_dict[file_name] = int(test_labels[i])
            cv2.imwrite(path,image_patch[j].astype(np.float64))

    with open(os.path.join(save_dir_splitted,"test","labels","test.json"), 'w') as f:
        json.dump(test_labels_dict, f)#labels
    with open(os.path.join(save_dir_splitted,"test","labels_patch","test_patch.json"), 'w') as f:
        json.dump(test_labels_patch_dict, f)#labels
        
    #VALIDATION
    os.makedirs(os.path.join(save_dir_splitted,"val","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"val","images_patch"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"val","labels"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"val","labels_patch"),exist_ok=True)
    
    val_labels_dict = {}
    val_labels_patch_dict = {}
    
    for i, image in enumerate(tqdm(val_data,desc = "Validation")):               #images
        file_name = str(i)+"_val"+".png"
        path = os.path.join(save_dir_splitted,"val","images",file_name)
        
        cv2.imwrite(path,image.astype(np.float64))
        val_labels_dict[file_name] = int(val_labels[i])
        
        image_patch = crop_image(image, patch_size, n_samples)
        for j in range(len(image_patch)):
            file_name = str(i)+"-"+str(j)+"_val"+".png"
            path = os.path.join(save_dir_splitted,"val","images_patch",file_name)
            val_labels_patch_dict[file_name] = int(val_labels[i])
            cv2.imwrite(path,image_patch[j].astype(np.float64))
        
    with open(os.path.join(save_dir_splitted,"val","labels","val.json"), 'w') as f:
        json.dump(val_labels_dict, f)#labels
    with open(os.path.join(save_dir_splitted,"val","labels_patch","val_patch.json"), 'w') as f:
        json.dump(val_labels_patch_dict, f)#labels

### Dataloaders definition

Notice that the labels are saved into a numpy array, while the images are read directly from the folder

In [166]:
class HDAdataset(Dataset):
    def __init__(self, root_dir, labels, train = False, transform=None, grayscale= False, resize_to= None, fft = False):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        self.grayscale=grayscale
        self.fft = fft
        self.resize_to = resize_to

    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        img_name = os.listdir(self.root_dir)[idx]
        img_path = os.path.join(self.root_dir, img_name)
        
        if self.grayscale or self.fft:
            image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
            
        else:
            image = cv2.imread(img_path)
        
        if self.train:
            image = image.astype(np.uint8) + np.random.normal(0, 2, image.shape).astype(np.uint8)
        
        if self.resize_to is not None:
            image = cv2.resize(image, resize_to, interpolation= cv2.INTER_LINEAR)
            
        if self.fft:
            image = np.fft.fft2(image)
            fshift = np.fft.fftshift(image)
            magnitude_spectrum = 20*np.log(np.abs(fshift))
            image = magnitude_spectrum
        

        
        #labels smoothing
        label = 0.1*np.ones(3)
        label[self.labels[img_name]] = 0.8
        
        if self.transform:
            image = self.transform((image/255.).astype(np.float32))

        return image, label

In [167]:
class HDAdatasetPatches(Dataset):
    def __init__(self, root_dir,patch_dir, labels, train = False, transform=None, grayscale= False, fft = False):
        self.root_dir = root_dir
        self.patch_dir = patch_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        self.grayscale=grayscale
        self.fft = fft

    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        img_name = os.listdir(self.root_dir)[idx]
        img_id = int(img_name.split("_")[0])
        
        patches_list = os.listdir(self.patch_dir)
        patches_image_list = list(filter(lambda name: int(name.split("-")[0]) == img_id, patches_list))
        
        all_patches = []
        for patch in patches_image_list:
            patch_path = os.path.join(self.patch_dir, patch)
            

            if self.grayscale or self.fft:
                image = cv2.imread(patch_path,cv2.IMREAD_GRAYSCALE)
                channels = 1
            
            else:
                image = cv2.imread(patch_path)
                channels = 3

            if self.train:
                image = image.astype(np.uint8) + np.random.normal(0, 2, image.shape).astype(np.uint8)

            if self.fft:
                image = np.fft.fft2(image)
                fshift = np.fft.fftshift(image)
                magnitude_spectrum = 20*np.log(np.abs(fshift))
                image = magnitude_spectrum
                
            all_patches.append(image.reshape(image.shape[0],image.shape[1],channels))

        all_patches=np.stack(all_patches)
        
        #labels smoothing
        label = 0.1*np.ones(3)
        label[self.labels[img_name]] = 0.8
        
        if self.transform:
            all_patches = self.transform((all_patches/255.).astype(np.float32))

        return all_patches, label

In [168]:
transform = transforms.Compose([transforms.ToTensor()]) 


In [169]:
if(LOAD_DATASET and not DATASET_ALREADY_DOWNLOADED):
    print("download the dataset")
    urllib.request.urlretrieve("", "dataset.zip")
    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("./dataset/")

#convert to tensor each image
transform = transforms.Compose([transforms.ToTensor()])    

#TRAIN DATASET AND DATALOADER
train_dir = os.path.join(data_dir,"train","images_patch")#images
train_labels_dir = os.path.join(data_dir,"train","labels_patch","train_patch.json")
with open(train_labels_dir) as f:
    train_labels = json.load(f)#labels

train_dataset = HDAdataset(train_dir,train_labels,train = True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

#TEST DATASET AND DATALOADER
test_dir = os.path.join(data_dir,"test","images")#images
test_dir_patches = os.path.join(data_dir,"test","images_patch")#patches
test_labels_dir = os.path.join(data_dir,"test","labels","test.json")
with open(test_labels_dir) as f:
    test_labels = json.load(f)#labels

test_dataset = HDAdatasetPatches(test_dir,test_dir_patches,test_labels,train = False,
                          transform=None,grayscale=grayscale,fft=fft)

#test_dataloader = DataLoader(test_dataset, batch_size=n_samples, shuffle=False)

#VALIDATION DATASET AND DATALOADER
val_dir = os.path.join(data_dir,"val","images")#images
val_dir_patches = os.path.join(data_dir,"val","images_patch")#patches
val_labels_dir = os.path.join(data_dir,"val","labels","val.json")
with open(val_labels_dir) as f:
    val_labels = json.load(f)#labels

val_dataset = HDAdatasetPatches(val_dir,val_dir_patches,val_labels,train = False,
                          transform=None,grayscale=grayscale,fft=fft)

#val_dataloader = DataLoader(val_dataset, batch_size=n_samples, shuffle=False)

In [170]:
for data in val_dataloader:
    im, lab = data
    print(lab)
    '''
    fig, ax = plt.subplots(nrows = 5, ncols = 8, figsize = (10, 8))
    for i in range(5):
        for j in range(8):
            ax[i, j].imshow(im[j+8*i, 0], cmap = 'Greys')
    '''

NameError: name 'val_dataloader' is not defined

## Neural network

In [171]:
torch.cuda.empty_cache()
gc.collect()

182

### Models definition

In [230]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            ### 6x128x128
            nn.MaxPool2d(2),
            ### 6x64x64
            nn.BatchNorm2d(6),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.4),
            nn.Conv2d(in_channels=6, out_channels=9, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            ### 9x22x22
            nn.MaxPool2d(2),
            ### 9x11x11
            nn.BatchNorm2d(9),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.4),
            nn.Conv2d(in_channels=9, out_channels=12, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            ### 12x4x4
            nn.MaxPool2d(2),
            ### 12x2x2
            nn.BatchNorm2d(12),
            nn.LeakyReLU(),
            nn.Flatten(),
            ### 12*2*2
            nn.LeakyReLU(),
            #nn.Dropout(p=0.2),
            nn.Linear(48,30),
            nn.BatchNorm1d(30),
            nn.ReLU(),
            #nn.Dropout(p=0.2),
            nn.Linear(30,20),
            nn.LeakyReLU(),
        )

        self.classification = nn.Sequential(
            nn.Linear(20,10),
            nn.BatchNorm1d(10),
            nn.ReLU(),
            #nn.Dropout(p=0.2),
            nn.Linear(10,3),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x, self.classification(x)
    
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        self.decoder = nn.Sequential(
            nn.Linear(20,30),
            nn.BatchNorm1d(30),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Linear(30,48),
            nn.BatchNorm1d(48),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Unflatten(-1, (12,2,2)),
            nn.Upsample(size=(4, 4), mode='bilinear'),
            nn.ConvTranspose2d(in_channels=12, out_channels=9, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.BatchNorm2d(9),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Upsample(size=(22, 22), mode='bilinear'),
            nn.ConvTranspose2d(in_channels=9, out_channels=6, kernel_size=(3,3), stride=(3,3), padding=(0,0)),
            nn.BatchNorm2d(6),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Upsample(size=(128, 128), mode='bilinear'),
            nn.ConvTranspose2d(in_channels=6, out_channels=3, kernel_size=(3,3), stride=(1,1), padding=(1,1), dilation=1,output_padding=(0,0)),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.decoder(x)
        
        return x

In [231]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(BOLD+"You are working on: "+END+str(device))

You are working on: cuda


### Loading of the models

In [232]:
if(LOAD_MODEL and not MODEL_ALREADY_DOWNLOADED):
    print("download the model")

In [233]:
if(not LOAD_MODEL):
    encoder = Encoder().to(device)
    decoder = Decoder().to(device)
    #classifier = Classifier().to(device)
    
else:
    print("load the models from the folder")

In [234]:
# inp = np.random.uniform(0,2,(3,60)).astype(np.float32)

In [235]:
# out = decoder(torch.tensor(inp).to(device))
# out.shape

In [236]:
summary(encoder, (3, 128, 128),batch_size=3)
summary(decoder, (20, ), batch_size=3)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [3, 6, 128, 128]             168
         MaxPool2d-2             [3, 6, 64, 64]               0
       BatchNorm2d-3             [3, 6, 64, 64]              12
         LeakyReLU-4             [3, 6, 64, 64]               0
            Conv2d-5             [3, 9, 22, 22]             495
         MaxPool2d-6             [3, 9, 11, 11]               0
       BatchNorm2d-7             [3, 9, 11, 11]              18
         LeakyReLU-8             [3, 9, 11, 11]               0
            Conv2d-9              [3, 12, 4, 4]             984
        MaxPool2d-10              [3, 12, 2, 2]               0
      BatchNorm2d-11              [3, 12, 2, 2]              24
        LeakyReLU-12              [3, 12, 2, 2]               0
          Flatten-13                    [3, 48]               0
        LeakyReLU-14                   

### Training 

The accuracy in the training is patch wise while the accuracy in validation and test is image wise(i.e. the most common class for the patches of the same image)

In [237]:
def test(dataset,name):
    
    losses_ae = []
    losses_c = []
    count_correct = 0
    count_input = 0

    encoder.eval()
    decoder.eval()
    #classifier.eval()
    for data in tqdm(dataset,desc = name +" set"):
        pred = []
        labels = data[1].to(device)
        for patch in tqdm(data):
            
            img = torch.tensor(patch[0]).to(device)
        
            compressed_img,pred_labels = encoder(img)
            rec_img = decoder(compressed_img)
            

            classifier_loss = criterion_classifier(pred_labels, labels)
            ae_loss= criterion_ae(img,rec_img)


            losses_ae.append(ae_loss.detach().cpu().numpy())
            losses_c.append(classifier_loss.detach().cpu().numpy())
            
            pred.append(pred_labels.to("cpu").detach().numpy().argmax(axis=1))
            
            
        count_input += 1
            
        final_image_label = max(set(pred), key=pred.count)
        true_image_label = labels.to("cpu").detach().numpy().argmax(axis=1)[0]
        count_correct+= (final_image_label == true_image_label)
        
        del img
        del labels
        torch.cuda.empty_cache()
        gc.collect();

    # Print loss at end of each epoch
    accuracy= (count_correct/count_input)*100.
    print(name,' Autoencoder Loss: %.4f, Classifier Loss: %.4f, Accuracy: %.4f in %d input samples\n'
      % (np.mean(losses_ae), np.mean(losses_c),accuracy,count_input))
    
    return np.mean(losses_ae),np.mean(losses_c),accuracy

In [238]:
criterion_classifier = nn.CrossEntropyLoss()
criterion_ae = nn.MSELoss()

def train(train_dataloader,val_dataset):
    optimizer_e = optim.Adam(encoder.parameters(), lr=learning_rate_e,weight_decay=1e-2)
    optimizer_d = optim.Adam(decoder.parameters(), lr=learning_rate_d,weight_decay=1e-2)

    
    
    best_epoch = 0 #(based on the validation accuracy)
    best_accuracy = -np.inf
    
    #for plotting
    train_loss_ae_trend = []
    train_loss_c_trend = []
    train_accuracy_trend =[]
    val_loss_ae_trend = []
    val_loss_c_trend = []
    val_accuracy_trend = []

    
    for epoch in range(num_epochs):

        ######training#####
        losses_ae = []
        losses_c = []
        count_correct = 0
        count_input = 0

        encoder.train()
        decoder.train()

        #train autoencoder for one epoch
        for data in tqdm(train_dataloader, desc = "Train autoencoder Epoch [%d/%d]"%(epoch+1,num_epochs)):

            img, labels = data
            img = img.to(device)
            labels = labels.to(device)
            
            optimizer_e.zero_grad()
            optimizer_d.zero_grad()

            compressed_img,pred_labels= encoder(img)
            rec_img = decoder(compressed_img)

            #classifier_loss = criterion_classifier(pred_labels, labels)
            ae_loss = criterion_ae(img,rec_img)
            ae_loss2 = criterion_ae(img,rec_img)
            
            ae_loss.backward()
            
            optimizer_e.step()
            optimizer_d.step()
            #total_loss = classifier_loss+0.01*ae_loss


#             ae_loss2.backward()
#             optimizer_e.step()
#             optimizer_d.step()

            losses_ae.append(ae_loss.detach().cpu().numpy())
#             losses_c.append(classifier_loss.detach().cpu().numpy())

        #train classifier for one epoch    
        for data in tqdm(train_dataloader,desc = "Train classifier Epoch [%d/%d]"%(epoch+1,num_epochs)):

            img, labels = data
            img = img.to(device)
            labels = labels.to(device)
            
            optimizer_e.zero_grad()
            #optimizer_d.zero_grad()

            compressed_img,pred_labels= encoder(img)
            #rec_img = decoder(compressed_img)

            classifier_loss = criterion_classifier(pred_labels, labels)
#             ae_loss = criterion_ae(img,rec_img)
#             total_loss = classifier_loss+0.01*ae_loss


            classifier_loss.backward()
            optimizer_e.step()
            #optimizer_d.step()
            


            #losses_ae.append(ae_loss.detach().cpu().numpy())
            losses_c.append(classifier_loss.detach().cpu().numpy())

            count_input+=pred_labels.detach().cpu().numpy().shape[0]
            count_correct+=np.sum((labels.to("cpu").detach().numpy().argmax(axis=1)==pred_labels.to("cpu").detach().numpy().argmax(axis=1)))
            del img
            del labels
            torch.cuda.empty_cache()
            gc.collect();
            
        accuracy = (count_correct/count_input)*100.
        
        print('TRAIN: Epoch [%d/%d], Autoencoder Loss: %.4f, Classifier Loss: %.4f, Accuracy: %.4f in %d input samples \n'
              % (epoch+1, num_epochs, np.mean(losses_ae), np.mean(losses_c),accuracy,count_input))
        
        train_loss_ae_trend.append(np.mean(losses_ae))
        train_loss_c_trend.append(np.mean(losses_c))
        train_accuracy_trend.append(accuracy)
        
        ######validation######
        ae,c,acc = test(val_dataset,"VALIDATION")
        
        val_loss_ae_trend.append(ae)
        val_loss_c_trend.append(c)
        val_accuracy_trend.append(acc)
        
        if acc > best_accuracy:
            print(GREEN+"Saved Model: "+END+UNDERLINE+"best validation accuracy reached\n\n"+END)
            best_epoch = epoch+1
            torch.save(encoder.state_dict(), "best_e.pt")
            torch.save(decoder.state_dict(), "best_d.pt")
            
            best_accuracy = acc

    return np.array([[train_loss_ae_trend,train_loss_c_trend,train_accuracy_trend],
            [val_loss_ae_trend,val_loss_c_trend,val_accuracy_trend],best_epoch],dtype=np.object_)

In [ ]:
if(not LOAD_MODEL):
    results = train(train_dataloader,val_dataset) 
    np.save("results.npy",results)

Train autoencoder Epoch [1/50]:  24%|███████████                                    | 247/1044 [00:43<02:25,  5.48it/s]

### Test

Notice that the value of the losses and the accuracy for the train set are computed during the training, so they are not very accurate.

In [ ]:
train_dataset_no_noise = HDAdataset(train_dir,train_labels,train = False, transform=transform)
train_dataloader_no_noise = DataLoader(train_dataset_no_noise, batch_size=64, shuffle=False,drop_last=False)

In [ ]:
test(train_dataloader_no_noise,"TRAIN")
test(val_dataloader,"VALIDATION")
test(test_dataloader,"TEST");